In [1]:
## for jupyter notebook only
import warnings

warnings.simplefilter("ignore")

from IPython.display import HTML, display

display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)


import json
import os

# Importing libraries
import pandas as pd
from ds.utilities.io import ds_trino

pd.options.display.max_columns = None
pd.options.display.max_rows = 50
pd.options.mode.chained_assignment = None


conn = ds_trino.create_trino_connection()

In [2]:
import numpy as np

In [3]:
query_uni = """

with scan_out as (SELECT
      CAST(order_uuid AS VARCHAR) AS order_id
      ,TRUE AS order_scanned_out
      ,count ((movimiento.uuid)) as scanned_transaction
      ,count(DISTINCT (sku_uuid)) as scanned_sku
      ,sum(abs(requested_stock_on_hand_delta)) as requested_scanned_items
      ,sum(abs(applied_stock_on_hand_delta)) as applied_scanned_items
      , sum(stock_on_hand_absolute) as absolute_scanned_items
      
      
    FROM
    hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction as movimiento 
    left join hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction_line_item as item
     on cast(movimiento.uuid as VARCHAR ) = item.inventory_transaction_uuid
    
    WHERE
      movimiento.day_partition>= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '7' month, '%Y-%m-%d')
      and item.day_partition>= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '7' month, '%Y-%m-%d')
      
      AND type = 'TRANSACTION_ORDER_SCANOUT'
      AND cause = 'CAUSE_MANUAL'
    GROUP BY
      1,2)



SELECT 
DATE(from_unixtime(reference_time/1000, facility_timezone)) AS created_date,
store_id,

date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_week_monday,

--que empiece en domingo
date_trunc('week', DATE(from_unixtime((reference_time)/1000, facility_timezone))+ interval '1' day)- interval '1' day as created_week,

FLOOR((DATE_DIFF('day', DATE(created_at_ts), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 3) / 7) + 1 AS order_weeks_back_local,
cast(local_created_at_ts as time) as local_created_time,

cast(local_ordered_at_ts as time) as local_ordered_at_ts,
facility_timezone, 
order_items_names,

days_since_last_order_via_internal_customer_id,catering_meal_segment, date_string_local,

handoff_bin_id,	handoff_bin_display_id,

dc.country, dc.region,facility_region, dc.sub_region,

(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
      facility_id in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
          '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
          '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
          '812e6913-d78e-4dbc-877d-16993853aa7e' -- sao domingos
      )
   then
      'Sao Paulo'
      
  WHEN 
        facility_id IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when facility_id = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN facility_id = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN facility_id = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN facility_id = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN facility_id = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN facility_id = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
   --WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when facility_id = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
    when facility_id = '812e6913-d78e-4dbc-877d-16993853aa7e' THEN 'SAO - Domingos'
  
   WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'
   else facility_name

  
END
) AS facility_sf_name,
facility_name,
organization_name as org_name,
organization_id,
brand_id,
brand_name,
co.order_id,
facility_id,
order_display_id,
ofo_slug, store_link_service_slug,service_slug,
facility_country_code as country_code, 
is_cancelled,
oc_order_issue_count,
order_issue_details,
hoi_order_issue_details,
oc_order_issue_details,
order_issue_types,
hoi_order_issue_types,
oc_order_issue_types,

subtotal,
total,

commission, discount,	tip, tip_for_restaurant, tip_for_courier, delivery_fee, marketing_fee, service_fee

,ROW_NUMBER() OVER (PARTITION BY ofo_slug, external_order_id,normalized_order_id ,external_order_display_id,order_items_quantity, subtotal ORDER BY co.order_id) as row_number

,cast(local_accepted_at_last_ts as TIME ) as accepted_local_time
,cast(AT_TIMEZONE(accepted_at_ts, facility_timezone) as TIMESTAMP ) as accepeted_at_local
,local_processed_at_ts,local_accepted_at_ts, local_in_handoff_at_ts,
local_handed_off_at_ts,
local_printed_at_ts,local_trashed_at_ts, printed_at,
local_pickup_picked_up_at_ts,
local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,normalized_order_id, external_order_display_id,
fulfillment_state,fulfillment_mode,
pickup_state,scheduling_type,ofo_status,order_status
,cancellation_state,cancellation_reason as facility_cancellation_reason
,ofo_cancellation_reason,order_rating, order_review_full_text,location_state,
delivery_service_provider_slug,

case when location_state='LOCATION_HANDED_OFF' then 1
  else 0
  end
as is_handed_off_num,

Case When in_handoff_at IS NOT NULL THEN 1
  else 0
  END 
as is_in_handoff_num,




CASE
        WHEN is_cancelled THEN 'cancel'
        WHEN order_issue_types IS NOT NULL THEN 'issue'
        ELSE 'success'
        END AS defect_type,

CASE
        WHEN is_cancelled AND cancellation_reason = 'UNAVAILABLE_ITEM' THEN 'item_unavailable'
        WHEN is_cancelled AND confirmation_state <> 'CONFIRMATION_CONFIRMED' or ofo_cancellation_reason='UNACCEPTED_ORDER' THEN 'confirmation_failed'
        WHEN is_cancelled AND  location_state!='LOCATION_HANDED_OFF' THEN 'handoff_failed'
        --WHEN is_cancelled AND time_to_last_in_handoff_millis > 1200000 THEN 'pick_over_20' --Pick time was over 20 minutes long
        WHEN is_cancelled THEN 'other_cancel'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ORDER%' THEN 'order_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%MISSING_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_QUANTITY%' THEN 'item_missing'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_SIZE%' THEN 'item_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%INCOMPLETE%' THEN 'order_incomplete'
        WHEN array_join(oc_order_issue_details,' ') LIKE '%canceled by the OFO%' THEN 'Canceled by OFO'
        WHEN array_join(oc_order_issue_details,' ') LIKE '%canceled by OFO%' THEN 'Canceled by OFO'
        WHEN array_join(oc_order_issue_details,' ') LIKE '%Re-request delivery%' THEN 'Courier not found'
        ELSE 'other_issue'
      END AS defect_reason,

COALESCE(IF(adjustment IS NOT NULL AND (is_cancelled OR order_issue_types IS NOT NULL), adjustment < 0), FALSE) AS is_refunds
,time_to_last_in_handoff_millis BETWEEN 1 AND 1200000 AS order_20picked --TRUE if picked in under 20 minutes

,order_source, hoi_only_order_type,order_items_quantity,scan_out.order_scanned_out,


printed_at_ts, in_handoff_at_ts, handed_off_at_ts, courier_checked_in_at_ts, courier_dropped_off_at_ts, courier_assigned_at_ts,
courier_arrived_at_ts, ordered_at_ts, cancelled_at, discovered_at,

cast(from_unixtime(discovered_at/1000, facility_timezone) as TIMESTAMP ) AS discovered_date_local,
cast(from_unixtime(discovered_at/1000, 'UTC') as TIMESTAMP ) AS discovered_date_utc,
has_inactive_onboarding_store_connection,
has_disabled_store_connection


--scanner
,scanned_transaction, scanned_sku, applied_scanned_items

,customer_note
,payment_due_payment_method
,tenant_type,destination_latitude, destination_longitude, facility_latitude, facility_longitude
,customer_id
,cloud_retail_order_type


,concat('https://tools.cloudkitchens.com/order-inspector/order-details/',co.order_id,'/overview') as link


FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
   
left join scan_out
on co.order_id=scan_out.order_id






--aqui se filtra la fecha    
WHERE 
--co.day_partition >= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '7' month, '%Y-%m-%d')
day_partition>='2020-01-01'
and reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-15'
AND tenant_type in ('VIRTUAL','CLOUD_RETAIL')
--and ofo_slug in ('rappishopper','rappi')
--AND organization_name='Tere Cazola'
and organization_id in ('0f04feb7-3919-48bb-a708-7d13d1738354')
--AND is_cancelled = False
--and co.order_id ='79a16606-61b2-46f5-b232-15c1c72b8d80'
and organization_id not in (

'45ad476b-9874-4513-8a29-1b3b04180fbd', --casa de pastor
'2f3aefc9-2f28-45df-8de5-2b52530abef1', --bento vk
'cb671780-a757-4db5-8782-100106cad446', --delichurros vk
'842a7a9f-e25c-433d-bb4f-c1912db1bb04', --guisamor
'8d09b777-805e-49cf-a172-27fbf2840dd1',--donas americanas
'46f35c4f-1bf1-479d-92c9-e729a40ba816', --FREE-TANGAS & GAR-NACHAS - VK


--tests:

'beb89975-2feb-4d0b-a61e-ac2a3b87bd18', --Org Test PiknPak
'24622819-678c-45fb-a5be-acece3bf9c73', -- CR Brazil - Teste Superstore
'b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a', -- Conta Teste - Catherine
'fae923c9-8a1d-468e-9740-95c36c0fd4fc', --Conta Teste Pick n pack - Comercial
'804c6336-abc2-4293-81ee-0cea88c15029' -- warehouse and ops analyst


)
--and co.order_id='a0881383-41ad-4b06-a993-6a69df010883'

AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
           -- 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            --'6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
    --ECUADOR
        -- QUITO
            --'d65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
            --medellin
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f',--de moda
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
    -- COSTA RICA
        -- SAN JOSE
            --'9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
           -- '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - av mexico
            
-- CHILE
        -- SANTIAGO
            --'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
             '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
            '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
             'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
            --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
            '14d9bb92-6376-4d69-adb6-64f97e4b3561',-- SAO - Ferreira
            '812e6913-d78e-4dbc-877d-16993853aa7e', --sao domingos
            
                
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                
                )

"""

print("querying ...")
unilever = ds_trino.fetch_data(conn=conn, sql_str=query_uni, use_cache=False)
print("pulled")

querying ...
pulled


In [4]:
unilever

,created_date,store_id,created_month,created_week_monday,created_week,order_weeks_back_local,local_created_time,local_ordered_at_ts,facility_timezone,order_items_names,days_since_last_order_via_internal_customer_id,catering_meal_segment,date_string_local,handoff_bin_id,handoff_bin_display_id,country,region,facility_region,sub_region,city,facility_sf_name,facility_name,org_name,organization_id,brand_id,brand_name,order_id,facility_id,order_display_id,ofo_slug,store_link_service_slug,service_slug,country_code,is_cancelled,oc_order_issue_count,order_issue_details,hoi_order_issue_details,oc_order_issue_details,order_issue_types,hoi_order_issue_types,oc_order_issue_types,subtotal,total,commission,discount,tip,tip_for_restaurant,tip_for_courier,delivery_fee,marketing_fee,service_fee,row_number,accepted_local_time,accepeted_at_local,local_processed_at_ts,local_accepted_at_ts,local_in_handoff_at_ts,local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,printed_at,local_pickup_picked_up_at_ts,local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,normalized_order_id,external_order_display_id,fulfillment_state,fulfillment_mode,pickup_state,scheduling_type,ofo_status,order_status,cancellation_state,facility_cancellation_reason,ofo_cancellation_reason,order_rating,order_review_full_text,location_state,delivery_service_provider_slug,is_handed_off_num,is_in_handoff_num,defect_type,defect_reason,is_refunds,order_20picked,order_source,hoi_only_order_type,order_items_quantity,order_scanned_out,printed_at_ts,in_handoff_at_ts,handed_off_at_ts,courier_checked_in_at_ts,courier_dropped_off_at_ts,courier_assigned_at_ts,courier_arrived_at_ts,ordered_at_ts,cancelled_at,discovered_at,discovered_date_local,discovered_date_utc,has_inactive_onboarding_store_connection,has_disabled_store_connection,scanned_transaction,scanned_sku,applied_scanned_items,customer_note,payment_due_payment_method,tenant_type,destination_latitude,destination_longitude,facility_latitude,facility_longitude,customer_id,cloud_retail_order_type,link
0,2023-08-23,b306006e-86bf-4b4a-948e-8e5a792966eb,2023-08-01,2023-08-21,2023-08-20,4,16:41:57.000,16:41:57.000,America/Sao_Paulo,"[Brigadeiro 77ml, Cornetto Crocante 90ml, Mag...",NaN,MEAL_SEGMENT_UNKNOWN,2023-08-23,None,None,Brazil,LATAM,Brazil,BRAZIL,Sao Paulo,SAO - Rua do Acre,BR - Sao Paulo - Acre,Unilever - Pik n' Pak,0f04feb7-3919-48bb-a708-7d13d1738354,44d3d289-faff-4d79-b923-1fccefd84239,Kibon Express,42a29ba4-0f54-363c-b2d9-34a9abf51ec4,6c601837-403c-425f-a51c-31428ed1a67e,6wxce,d2c-eater-website,d2c-eater-website,d2c-eater-website,BR,True,1.0,None,None,[Order has been canceled by the OFO. Do not pr...,None,None,[CANCELED],2315.49,2315.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1,16:41:58.000,2023-08-23 16:41:58.000,2023-09-08 20:26:19.000 America/Sao_Paulo,2023-08-23 16:41:58.000 America/Sao_Paulo,None,None,None,2023-08-23 18:30:39.000 America/Sao_Paulo,NaN,None,None,None,1079fbca-7135-4f8d-9110-2f5567a0b7c9,1079fbca-7135-4f8d-9110-2f5567a0b7c9,wn5h1,FULFILLMENT_NOT_FULFILLED,FULFILLMENT_MODE_RESTAURANT_DELIVERY,PICKUP_NOT_PICKED_UP,SCHEDULING_ASAP,OFO_STATUS_CANCELED,OFO_STATUS_CANCELED,CANCELLATION_CONFIRMED,REASON_UNKNOWN,None,NaN,None,LOCATION_TRASHED,None,0,0,cancel,handoff_failed,False,None,NIO,None,14,None,None,None,None,None,None,None,None,2023-08-23 19:41:57.000,1.692826e+12,1.692820e+12,2023-08-23 16:41:57.000,2023-08-23 19:41:57.000,True,False,NaN,NaN,NaN,verificar se o estabelecimento estará aberto.,CASH,CLOUD_RETAIL,-23.567204,-46.5746827,-23.559152603149414,-46.58283233642578,a04ce4b5-95ed-4f83-b8b3-3804d609dfe8,DIRECT,https://tools.cloudkitchens.com/order-inspecto...
1,2023-09-05,b306006e-86bf-4b4a-948e-8e5a792966eb,2023-09-01,2023-09-04,2023-09-03,2,11:04:39.000,11:04:39.000,America/Sao_Paulo,"[Brigadeiro 77ml, Chicabon Zero 60ml, Fruttare...",NaN,MEAL_SEGMENT_UNKNOWN,2023-09-05,94765c4c-f009-463b-956f-ae22150d5cea,A1,Brazil,LATAM,Brazil,BRAZIL,Sao Paulo,SAO - Rua do Acre,BR - Sao Paulo

In [5]:
#importo secreto
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pygsheets
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
s = secrets.Secrets()
credentials_sheets = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"),scopes=SCOPES)
#gc 

In [6]:
# movimientos

## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1aaCEmz91Aj7FrGiAPsS0eYf7_xXjuTbMN4G77owuQO0')

# select sheet

ws = ss.worksheet('title','cr_orders')

print('Clearing Range ...')
ws.clear(start='A', end='Z')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(unilever, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!
